# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import azureml.core
import json
from azureml.core import Workspace, Experiment, Model
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
import joblib 
from azureml.train.automl import AutoMLConfig
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

print("SDK version:", azureml.core.VERSION)

SDK version: 1.26.0


### Initialize workspace

In [2]:
ws = Workspace.from_config()

print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-143097
aml-quickstarts-143097
southcentralus
9b72f9e6-56c5-4c16-991b-19c652994860


### Create an Azure ML experiment

In [3]:
# choose a name for experiment
experiment_name = 'heart-failure-experiment'

experiment = Experiment(ws, experiment_name)

run = experiment.start_logging()

## Dataset

### Overview
This task is to predict mortality caused by heart disease. 

Attribute information:

age - Age of the patient<br />
anemia - Decrease of red blood cells or hemoglobin (boolean)<br />
creatinine_phosphokinase - Level of the CPK enzyme in the blood (mcg/L)<br />
diabetes - If the patient has diabetes (boolean)<br />
ejection_fraction - Percentage of blood leaving the heart at each contraction (percentage)<br />
high_blood_pressure - If the patient has hypertension (boolean)<br />
platelets - Platelets in the blood (kiloplatelets/mL)<br />
serum_creatinine - Level of serum creatinine in the blood (mg/dL)<br />
serum_sodium - Level of serum sodium in the blood (mEq/L)<br />
sex - Woman or man (binary)<br />
smoking - If the patient smokes or not (boolean)<br />
time - Follow-up period (days)<br />
DEATH_EVENT - If the patient deceased during the follow-up period (boolean)<br />

In [4]:
amlcompute_cluster_name = "project-compute"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Cluster exists.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Cluster exists.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
key = "heart-disease-database"

dataset = ws.datasets[key]

df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


In [6]:
dataset.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [9]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "iterations": 40,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 3,
    "primary_metric" : 'accuracy'
}
automl_config = AutoMLConfig(compute_target = 'project-compute',
                            task = "classification",
                            training_data=dataset,
                            label_column_name="DEATH_EVENT",
                            enable_early_stopping= True,
                            debug_log = "automl_errors.log",
                            **automl_settings
                            )

In [10]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on project-compute with default configuration
Running on remote compute: project-compute


Experiment,Id,Type,Status,Details Page,Docs Page
heart-failure-experiment,AutoML_b428813d-4f10-47e5-8828-f7c82f118840,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Beginning model selection.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

*************************************************************************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
heart-failure-experiment,AutoML_b428813d-4f10-47e5-8828-f7c82f118840,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_b428813d-4f10-47e5-8828-f7c82f118840',
 'target': 'project-compute',
 'status': 'Completed',
 'startTimeUtc': '2021-04-19T02:06:39.234909Z',
 'endTimeUtc': '2021-04-19T02:23:16.539768Z',
 'properties': {'num_iterations': '40',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'project-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"heart-failure-experiment","subscription_id":"9b72f9e6-56c5-4c16-991b-19c652994860","resource_group":"aml-quickstarts-143097","workspace_name":"quick-starts-ws-143097","region":"southcentralus","compute_target":"project-compute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":40,"primary_metric":"accuracy","task_type":"classification","data_script":null,"test_size":0.0,"v

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [15]:
best_auto_run, fitted_model = remote_run.get_output()

In [17]:
best_metrics = best_auto_run.get_metrics()

print("Best Run Id : {}".format(best_auto_run.id), 
      "Accuracy : {}".format(best_metrics['accuracy']), 
      "Best metrics : {}".format(best_metrics), 
      "Best model : {}".format(fitted_model), sep = '\n')

Best Run Id : AutoML_b428813d-4f10-47e5-8828-f7c82f118840_38
Accuracy : 0.8695622895622895
Best metrics : {'precision_score_micro': 0.8695622895622895, 'accuracy': 0.8695622895622895, 'norm_macro_recall': 0.6664013767462044, 'recall_score_weighted': 0.8695622895622895, 'precision_score_weighted': 0.8745604228503799, 'log_loss': 0.4108024938438826, 'f1_score_macro': 0.8428151252963195, 'precision_score_macro': 0.8692977842166453, 'weighted_accuracy': 0.8983020161762952, 'recall_score_micro': 0.8695622895622895, 'f1_score_micro': 0.8695622895622895, 'average_precision_score_micro': 0.9062746273313508, 'average_precision_score_weighted': 0.9058815611235884, 'f1_score_weighted': 0.8650615728175429, 'AUC_micro': 0.9057287827772676, 'AUC_weighted': 0.8996966917656574, 'recall_score_macro': 0.8332006883731022, 'balanced_accuracy': 0.8332006883731022, 'average_precision_score_macro': 0.8807063760708056, 'matthews_correlation': 0.7004934744211296, 'AUC_macro': 0.8996966917656573, 'accuracy_tabl

In [18]:
print(fitted_model._final_estimator)

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('23',
                                           Pipeline(memory=None,
                                                    steps=[('standardscalerwrapper',
                                                            <azureml.automl.runtime.shared.model_wrappers.StandardScalerWrapper object at 0x7f3e00f733c8>),
                                                           ('randomforestclassifier',
                                                            RandomForestClassifier(bootstrap=True,
                                                                                   ccp_alpha=0.0,
                                                                                   class_weight=None,
                                                                                   criterion='gini',
                                                                                   max_depth=None,
  

In [19]:
model = best_auto_run.register_model(model_name = "heart-failure-best-model-automl", model_path = './outputs/model.pkl')
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-143097', subscription_id='9b72f9e6-56c5-4c16-991b-19c652994860', resource_group='aml-quickstarts-143097'), name=heart-failure-best-model-automl, id=heart-failure-best-model-automl:1, version=1, tags={}, properties={})


In [20]:
for model in Model.list(ws):
    print(model.name, 'version:', model.version)

heart-failure-best-model-automl version: 1


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [34]:
import os
os.makedirs('./automl-model', exist_ok=True)

best_auto_run.download_file('/outputs/model.pkl',os.path.join('./automl-model','heart-failure-best-model-automl.pkl'))

for f in best_auto_run.get_file_names():
    if f.startswith('outputs'):
        output_file_path = os.path.join('./aml-model', f.split('/')[-1])
        print(f'Downloading from {f} to {output_file_path} ...')
        best_auto_run.download_file(name=f, output_file_path=output_file_path)

TODO: In the cell below, send a request to the web service you deployed to test it.

In [35]:
model=best_auto_run.register_model(
            model_name = 'heart-failure-best-model-automl', 
            model_path = './outputs/model.pkl'
)

In [36]:
best_auto_run.download_file('outputs/conda_env_v_1_0_0.yml', 'conda_env.yml')
environment = Environment.from_conda_specification(name = 'heart-failure-env', file_path = 'conda_env.yml')

In [37]:
best_auto_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

In [38]:
inference_config = InferenceConfig(entry_script= 'score.py', environment=environment)

In [39]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, auth_enabled=True, enable_app_insights=True)

In [40]:
webservice = Model.deploy(workspace=ws, 
                        name='heart-failure-ws', 
                        models=[model], 
                        inference_config=inference_config,
                        deployment_config=aci_config,
                        overwrite=True)

In [41]:
webservice

AciWebservice(workspace=Workspace.create(name='quick-starts-ws-143097', subscription_id='9b72f9e6-56c5-4c16-991b-19c652994860', resource_group='aml-quickstarts-143097'), name=heart-failure-ws, image_id=None, compute_type=None, state=ACI, scoring_uri=Healthy, tags=http://8a191047-f0d5-45b6-b958-2c13f6a64159.southcentralus.azurecontainer.io/score, properties=None, created_by={})

In [42]:
# wait for deployment to finish and display the scoring uri and swagger uri
webservice.wait_for_deployment(show_output=True)

print("Status : {}".format(webservice.state), 
      "scoring_uri : {}".format(webservice.scoring_uri), 
      "swagger_uri : {}".format(webservice.swagger_uri), sep = '\n')

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-19 03:24:38+00:00 Creating Container Registry if not exists.
2021-04-19 03:24:41+00:00 Use the existing image.
2021-04-19 03:24:41+00:00 Generating deployment configuration.
2021-04-19 03:24:42+00:00 Submitting deployment to compute.
2021-04-19 03:24:45+00:00 Checking the status of deployment heart-failure-ws..
2021-04-19 03:30:14+00:00 Checking the status of inference endpoint heart-failure-ws.
Succeeded
ACI service creation operation finished, operation "Succeeded"
Status : Healthy
scoring_uri : http://8a191047-f0d5-45b6-b958-2c13f6a64159.southcentralus.azurecontainer.io/score
swagger_uri : http://8a191047-f0d5-45b6-b958-2c13f6a64159.southcentralus.azurecontainer.io/swagger.json


In [43]:
from pprint import pprint

# select samples from the dataframe
test_data = df.sample(5)
test_label = test_data.pop('DEATH_EVENT')

# convert the records to a json data file
data_json = test_data.to_dict(orient='records')

data = json.dumps({'data': data_json})
pprint(data)

('{"data": [{"age": 72.0, "anaemia": 1, "creatinine_phosphokinase": 110, '
 '"diabetes": 0, "ejection_fraction": 25, "high_blood_pressure": 0, '
 '"platelets": 274000.0, "serum_creatinine": 1.0, "serum_sodium": 140, "sex": '
 '1, "smoking": 1, "time": 65}, {"age": 55.0, "anaemia": 0, '
 '"creatinine_phosphokinase": 66, "diabetes": 0, "ejection_fraction": 40, '
 '"high_blood_pressure": 0, "platelets": 203000.0, "serum_creatinine": 1.0, '
 '"serum_sodium": 138, "sex": 1, "smoking": 0, "time": 233}, {"age": 60.0, '
 '"anaemia": 0, "creatinine_phosphokinase": 166, "diabetes": 0, '
 '"ejection_fraction": 30, "high_blood_pressure": 0, "platelets": 62000.0, '
 '"serum_creatinine": 1.7, "serum_sodium": 127, "sex": 0, "smoking": 0, '
 '"time": 207}, {"age": 75.0, "anaemia": 0, "creatinine_phosphokinase": 99, '
 '"diabetes": 0, "ejection_fraction": 38, "high_blood_pressure": 1, '
 '"platelets": 224000.0, "serum_creatinine": 2.5, "serum_sodium": 134, "sex": '
 '1, "smoking": 0, "time": 162}, {"ag

In [44]:
# get inference
output = webservice.run(data)
print(output)

{"result": [1, 0, 1, 1, 0]}


In [45]:
webservice.get_logs()

'2021-04-19T03:29:35,769729200+00:00 - gunicorn/run \n2021-04-19T03:29:35,771207100+00:00 - iot-server/run \n2021-04-19T03:29:35,789657000+00:00 - rsyslog/run \n2021-04-19T03:29:35,791414100+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_5a33d0b1846b3717ef969a5f35fc31b7/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_5a33d0b1846b3717ef969a5f35fc31b7/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_5a33d0b1846b3717ef969a5f35fc31b7/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_5a33d0b1846b3717ef969a5f35fc31b7/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_5a33d0b1846b3717ef969a5f35fc31b7/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

In [46]:
# select samples from the dataframe
test_data = df.sample(10)
test_label = test_data.pop('DEATH_EVENT')

# convert the records to a json data file
data_json = test_data.to_dict(orient='records')

data = json.dumps({'data': data_json})

# get inference
output = webservice.run(data)
print(output)

{"result": [0, 0, 1, 1, 0, 0, 1, 0, 0, 0]}


In [47]:
webservice.get_logs()

'2021-04-19T03:29:35,769729200+00:00 - gunicorn/run \n2021-04-19T03:29:35,771207100+00:00 - iot-server/run \n2021-04-19T03:29:35,789657000+00:00 - rsyslog/run \n2021-04-19T03:29:35,791414100+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_5a33d0b1846b3717ef969a5f35fc31b7/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_5a33d0b1846b3717ef969a5f35fc31b7/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_5a33d0b1846b3717ef969a5f35fc31b7/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_5a33d0b1846b3717ef969a5f35fc31b7/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_5a33d0b1846b3717ef969a5f35fc31b7/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

In [48]:
webservice.delete()
compute_target.delete()